# Business Analytics und Künstliche Intelligenz
Wintersemester 2023/2024

Prof. Dr. Jürgen Bock

### Lernziele
* Sie sind in der Lage den Unterschied zwischen Modellparametern und Hyperparametern zu erläutern und einige Hyperparameter aufzuzählen.
* Sie sind in der Lage wesentliche Schritte beim Vorbereiten von Datensätzen darzustellen, und können mehrere Methoden anwenden um sich einen Überblick über einen Datensatz zu verschaffen.
* Sie können Herangehensweisen beim Kodieren der Ausagbe von Multi-Class Klassifikatornetzen skizzieren, können das Prinzip geeigneter Aktivierungsfunktionen darstellen, und sind in der Lage die Ausgaben dieser Aktivierungsfunktionen zu interpretieren.
* Sie sind in der Lage neuronale Netze zur Multi-Class Klassifikation anzuwenden und die Klassifikationsergebnisse anhand geegneter Metriken zu evaluieren.
* Sie können das Grundprinzip von Convolutional Neural Networks zur Bildklassifikation wiedergeben.

## Parameter von Neuronalen Netzen

Es werden grundsätzlich zwei Arten von Parametern unterschieden:
- Modellparameter
- Hyperparameter

**Modellparameter** sind die Parameter, die das eigentliche (gelernte) Modell ausmachen. Im Falle von Neuronalen Netzen sind dies die **Gewichte**. Wurde bspw. das neuronale Netz traininert um Katzen auf Bildern zu erkennen, so charakterisieren die Modellparameter (Gewichte) das Modell von Katzenbildern.

**Hyperparameter** sind die Paremeter, die die Struktur des Modells und des Trainingsverfahrens charakterisieren. Alle Stellschrauben an denen man beim Trainineren drehen kann sind Hyperparameter. Dazu zählen:
- Struktur des neuronalen Netzes
  - Anzahl der Layers
  - Anzahl der Neuronen pro Layer
  - Art der Vernetzung (fully connected, convolutional, etc.)
- Anzahl der Trainingsepochen
- Stapelgröße im Batchtraining
- Wahl der *loss function*
- Wahl des Optimierers
- Learning Rate
- Codierung des Ergebnisses im Ausgangsvektor
- ...

Die Wahl der Hyperparameter ist eine der schwierigsten Aufgaben beim Training von neuronalen Netzen. Oftmals sind die Auswirkungen bestimmter Hyperparameter nicht systematisch zu bestimmen. Des weiteren sind Wechselwirkungen zwischen den Hyperparametern möglich.

## Künstliche Neuronale Netze zur Bilderkennung

Wir werden uns noch ein paar Hyperparameter genauer anschauen, indem wir und mit einer prominenten Anwendung von neuronalen Netzen beschäftigen: Der Bilderkennung.

Genauer gesagt handelt es sich dabei um die Klassifikation des Bildinhalts.

### Bilddaten

Bilddaten sind deutlich komplexer als die bisher betrachteten künstlich generierten Daten, oder die einfachen Klassifikationsdatensätze, wie *iris*.

In klassischen Bilderkennungsverfahren wurde in einem Vorverarbeitungsschritt eine gewisse Menge an Bildmerkmalen detektiert und extrahiert. (Kanten, Ecken, etc.)

In neuronalen Netzen wird jedes Pixel des Bildes als Merkmal betrachtet. Entsprechend groß ist der Eingangsvektor des neuronalen Netzes.

Beispielsweise für ein Bild der Größe 32 x 32 Pixel in 3 Farbkanälen benötigen wir einen Eingangsvektor der Größe

In [1]:
32*32*3

3072

Diese Möglichkeit wurde erst durch die jüngsten Fortschritte im Bereich neuronaler Netze geschaffen:
- Verfügbarkeit von großen Datenmengen
- Verfügbarkeit von performanter Rechenleistung
- Neuartige / verbesserte Algorithmen

Durch diese Fortschritte ist es möglich, sehr große neuronale Netze mit vielen Schichten mit einer Vielzahl von Neuronen - und auch entsprechend großer Eingangsvektoren - zu definieren und zu trainieren. Das Trainieren und Auswerten von Modellen mit solchen vielschichtigen Architekturen wird als *Deep Learning* bezeichnet.

#### Vorbereiten und Bereitstellen von Bilddaten mit PyTorch / Torchvision

Die `torchvision` Bibliothek enthält verschiedene nützliche Pakete und Module zum Laden, Verwalten und Manipulieren von Bilddaten.

Das Paket `datasets` in ``torchvision`` enthält Module zum Laden von frei verfügbaren und häufig (z.B. für Benchmarks) verwendeten Bilddatensätzen.

In [2]:
from torchvision import datasets

Das Paket `transforms` enthält Klassen zur Transformation von Bilddaten (z.B. Umwandlung in Tensoren, Umskalierung, Normalisierung, Zuschneiden, ...). Transformationen können einerseits verwendet werden, um die Bilder in ein für das neuronale Netz verwendbares Format zu konvertieren. Andererseits lassen sich damit Trainingsdaten erweitern, indem z.B. verschiedene Bildanschnitte verwendet werden, Bilder gedreht, gespiegelt, verzerrt, mit verschiedenen Sättigungs- / Kontrast- / Helligkeitsveränderungen modifiziert werden.

In [3]:
from torchvision import transforms

Die Transformationen können direkt der Repräsentation des Datensatzes übergeben werden. Sie werden dann vom Datensatzobjekt angewendet.

Zunächst muss jedoch ein Transformationsobjekt konfiguriert und instanziiert werden. Die Klasse `Compose` übernimmt bei der Instanziierung eine Liste von `transform` Objekten, die der Reihe nach ausgeführt werden.

In [4]:
transformations = transforms.Compose([
    transforms.ToTensor()
])

`torchvision` kann bekannte Datensätze bei Bedarf direkt herunterladen und in einem angegebenen Verzeichnis abspeichern

In [5]:
root_dir_cifar100 = "c:/data/cifar100/"

Sind die Daten am angegebenen Ort bereits vorhanden, wird der Download übersprüngen.

Je nach Datensatz stehen verschiedene Möglichkeiten zur Verfügung, z.B. wenn der Datensatz Test- und Trainingssätze enthält.

In [6]:
dataset_cifar100_train = datasets.CIFAR100( 
    root=root_dir_cifar100,
    train=True,
    download=True,
    transform=transformations)

8.0%


KeyboardInterrupt: 

In [ ]:
dataset_cifar100_test = datasets.CIFAR100( 
    root=root_dir_cifar100,
    train=False,
    download=True,
    transform=transformations)

Aus Lesbarkeitsgründen verwenden wir kürzere Bezeichner:

In [ ]:
data_train = dataset_cifar100_train
data_test = dataset_cifar100_test

#### Inspektion der Daten

Es ist immer sinnvoll, eine erste manuelle Inspektion der Daten vorzunehmen. Dadurch können wir prüfen, ob die Daten im richtigen Format vorliegen, ob sie gelesen und weiterverarbeitet werden können, und wie ggf. bestimmte Hyperparameter eingestellt werden müssen.

Handelt es sich um einen Trainings- oder Testdatensatz?

In [ ]:
print('data_train ist ein Trainingsdatensatz:', data_train.train)
print('data_test ist ein Trainingsdatensatz:', data_test.train)

In welcher Form liegen die Daten vor?

In [ ]:
print('Shape: ', data_train.data.shape)

Das bedeutet: 50000 Datensamples der Größe 32 x 32 x 3 (d.h. 32 x 32 Pixel in 3 Farbkanälen)

Wieviele und welche Klassen sind vorhanden?

In [ ]:
print("Anzahl der Klassen: ", len(data_train.classes))

for i in range(0, len(data_train.classes)):
    print("{:2d}  {}".format(i, data_train.classes[i]))

Wieviele Datensamples gibt es?

In [ ]:
print("Number of samples in training set:", len(data_train.data))
print("Number of samples in test set: ", len(data_test.data))

#### Filtern und Laden der Daten

Der Zugriff auf die Daten erfolgt, wie wir bereits wissen, über den `DataLoader` der im `pytorch` Paket `torch.utils.data` bereitgestellt wird.

In [ ]:
from torch.utils.data import DataLoader

In unserem Beispiel wollen wir nicht den gesamten Datensatz betrachten, sondern nur einen Teil, der bestimmte Target-Klassen enthält. Dazu können wir einen sogenannten `Sampler` konfigureren, der nur eine bestimmte Teilmenge des Datensatzes herausfiltert. Dem Sampler müssen die Indizes der in Frage kommenden Datensamples bekannt gemacht werden. Dazu müssen wir die Indizes dieser Datensamples der entsprechenden Target-Klassen zunächst herausfinden.

Wir wählen die gewünschten Klassen anhand ihrer Bezeichnungen aus und ermitteln die Klassenindizes:

In [ ]:
class_selection = ['apple', 'pear', 'orange', 'mushroom', 'sweet_pepper']
class_selection_idx = [i for i in range(len(data_train.classes)) if data_train.classes[i] in class_selection]

In [ ]:
print("Indizes der Klassen", class_selection, ":", class_selection_idx)

Wir benötigen nun die Indizes der Datensamples, welche mit den ausgewählten Klassen gelabelt sind (für welche das Target einer der gewählten Klassenindizes ist).

Inspizieren wir dazu zunächst den gesamten Target-Vektor:

In [ ]:
print(data_train.targets)

Wir benötigen die Indizes des Target-Vektors, an denen er eine der ausgewählten Klassen benennt:

In [ ]:
target_idx = [i for (i,t) in enumerate(data_train.targets) if t in class_selection_idx]

Hier sind die Indizes:

In [ ]:
print(target_idx)

Und hier zur Kontrolle der Teil-Target-Vektor selbst:

In [ ]:
print([data_train.targets[i] for i in target_idx])

Nun können wir mit Hilfe der Indexliste den `Sampler` erstellen. Die Samplerklassen finden sich im `torch.utils.data.sampler` Package. Wir verwenden den `SubsetRandomSampler` auf Basis der Indizes, die das Subset charakterisieren.

In [ ]:
from torch.utils.data.sampler import SubsetRandomSampler

In [ ]:
data_sampler = SubsetRandomSampler(target_idx)

Der eigentliche `DataLoader` kann nun für das `dataset` unter Angabe der `batch_size` und des `sampler`s erstellt werden.

In [ ]:
data_loader = DataLoader(dataset=data_train, batch_size=50, sampler=data_sampler)

Mit dem ``DataLoader`` können wir jetzt erstmals über die Bilder iterieren und einen Blick darauf werfen. Da wir unseren ``SubsetRandomSampler`` verwenden, sehen wir nur Bilder der ausgewählten Klassen.

Im bereitgestellten ``dataview`` Modul - dieses sollte im selben Verzeichnis wie dieses Notebook liegen - gibt es eine Hilfsfunktion zum Anzeigen von Bildern.

In [ ]:
import dataview

Wir zeigen die Bilder *batch*-weise an, wie wir sie vom ``DataLoader`` bekommen.

In [ ]:
for (input, _) in data_loader:
    dataview.view_images(input, 10)

### Das Neuronale Netz

#### Eingangs- und Ausgangsvektor

Wir definieren zunächst die Länge des Eingangsvektors: (32x32 Pixel in 3 Farbkanälen)

In [ ]:
n_input = 32 * 32 * 3

Bisher hatten wir neuronale Netze nur zur Binärklassifikation betrachtet. Dabei bestand die letzte Schicht aus genau einem Neuron, welches (bedingt durch die *sigmoid* oder *threshold* Aktivierungsfunktion Werte (nahe) 0 und (nahe) 1 ausgibt, was den beiden Klassen entspricht.

In diesem Beispiel haben wir es mit einer Multiclass Klassifikation zu tun, d.h. es gibt mehr als 2 Klassen.

Wir benötigen eine Methode um die verschiedenen Klassen als Ausgabe des neuronalen Netzes zu repräsentieren.

Es gibt zwei typische Ansätze, um Klassen zu kodieren:
- Label Encoding
- One-Hot Encoding

Beim **Label Encoding** wird jede Klasse durch eine eindeutigen Klassenindex repräsentiert. In unserem Falle wäre dies z.B.

| Klasse       | Label |
|--------------|-------|
| Apple        | 0     |
| Pear         | 1     |
| Orange       | 2     |
| Mushroom     | 3     |
| Sweet_Pepper | 4     |

Ein Vorteil ist die kompakte Repräsentation der Klasse. Ein Nachteil ist, dass durch die aufsteigende Nummerierung eine Reihenfolge suggeriert wird, die nicht vorhanden ist. Insbesondere, wenn das Klassenlabel später als numerischer Eingang zur Weiterverarbeitung verwendet wird könnte der Zahlenwert missinterpretiert werden.

Beim **One-Hot Encoding** wird für $n$ Klassen ein $n$-elementiger Vektor zur Repräsentation verwendet. In diesem Vektor gibt der Index die jeweilige Klasse an. Nach eindeutiger Klassifikation ist der Vektor in allen Elementen 0, außer in dem Element mit Index der erkannten Klasse ist er 1. In unserem Beispiel wäre das

| Klasse       | Apple | Pear | Orange | Mushroom | Sweet_Pepper |
|--------------|-------|------|--------|----------|--------------|
| Apple        | 1     | 0    | 0      | 0        | 0            |
| Pear         | 0     | 1    | 0      | 0        | 0            |
| Orange       | 0     | 0    | 1      | 0        | 0            |
| Mushroom     | 0     | 0    | 0      | 1        | 0            |
| Sweet_Pepper | 0     | 0    | 0      | 0        | 1            |

Die Verwendung des One-Hot Encodings für die Repräsentation des Ergebnisvektors neuronaler Netze bei Multiclass Klassifikation besitzt einen entscheidenden Vorteil gegenüber des Label Encodings: Das neuronale Netz liefert in der Regel keine eindeutige Klassenzuordnung mit einem Vektorelement gleich 1 und allen anderen gleich 0. Vielmehr werden die entsprechenden Ausgangsneuronen unterschiedlich stark aktiviert. Je stärker eines dieser Neuronen aktiviert ist, umso größer die Wahrscheinlichkeit für die entsprechende Klasse. (Eine Ergebnisdarstellung, die sich durch das Label Encoding nicht realisieren lässt.)

Für Multiclass Klassifikation gilt es also zunächst ein neuronales Netz zu konstruieren dessen *output layer* soviele Neuronen wie Zielklassen aufweist. Um die Aktivierungen in eine entsprechende Wahrscheinlichkeitsverteilung umzuwandeln  kann die *Softmax* Aktivierungsfunktion verwendet werden. Diese konvertiert einen Vektor $\vec{a}$ von Aktivierungen in einen gleichgroßen Vektor von Wahrscheinlichkeiten $\vec{y}$, so dass für jedes Vektorelement $y_i$ gilt $y_i \in [0, 1]$ und $\sum_i y_i = 1$.

<img src="softmax.png" width="400">

In [ ]:
import torch
import torch.nn as nn

In [ ]:
a = torch.randn(1, 5)
print('Aktivierungen a:                      ', a)

In [ ]:
softmax = nn.Softmax(dim=1)
print('Wahrscheinlichkeiten nach softmax(a): ', softmax(a))
print('Summe der Wahrscheinlichkeiten:       ', torch.sum(softmax(a)))

Eine häufige Verwendung findet auch die *LogSoftmax* Aktivierungsfunktion. Diese logarithmiert die Ergebnisse der *Softmax*-Funktion.

Die *LogSoftmax*-Funktion liefert Werte im Bereich $(-\infty, 0]$. Dies ist durch die Logarithmus-Funktion zu erklären:

In [ ]:
import matplotlib.pyplot as plt
import math
%matplotlib inline
xdata = torch.linspace(0, 1.5)

In [ ]:
plt.plot(xdata, torch.log(xdata))
plt.grid(True)
plt.axvline(1, color="red", linestyle="--")
plt.axvline(0, color="red", linestyle="--")
plt.show()


Dadurch werden höhere Wahrscheinlichkeiten (nahe 1) stärker betont (nach der Logarithmierung nahe 0) und geringe Wahrscheinlichkeiten (nahe 0) weiter abgeschwächt (nach der Logarithmierung nahe $-\infty$).

In [ ]:
logsoftmax = nn.LogSoftmax(dim=1)
print('Aktivierungen a:                      ', a)
print('Wahrscheinlichkeiten nach softmax(a): ', softmax(a))
print('Aktivierungen nach logsoftmax(a):     ', logsoftmax(a))

In [ ]:
plt.bar(torch.arange(len(a.flatten())) -0.3, a.flatten(), 0.2, label="Aktivierung")
plt.bar(torch.arange(len(a.flatten())) -0.1, softmax(a).flatten(), 0.2, label="Softmax")
plt.bar(torch.arange(len(a.flatten())) +0.1, logsoftmax(a).flatten(), 0.2, label="LogSoftmax")
plt.title("Vergleich Aktivierungsfunktionen")
plt.legend()
plt.show()

Ob mit *Softmax* oder *LogSoftmax* Aktivierung - die Länge des Ausgangsvektors des neuronalen Netzes entspricht der Anzahl der Zielklassen. In unserem Beispiel ist dies:

In [ ]:
n_output = len(class_selection)

Somit haben wir:

In [ ]:
print(n_input)
print(n_output)

#### Netzstruktur

Wir definieren zunächst ein klassisches *Multi-Layer Perceptron* wie bekannt.

In [ ]:
class MLP(nn.Module):
    
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(n_input, 200)
        self.fc2 = nn.Linear(200, 50)
        self.fc3 = nn.Linear(50, n_output)
        
    def forward(self, input):
        x = input.view(-1, n_input)
        x = torch.sigmoid(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        x = self.fc3(x)
        return x

Instanziierung des Modells:

In [ ]:
model = MLP()

#### Optimierer und *loss function*

Wir verwenden den performanten Optimierungsalgorithmus *Adam* zur Anpassung der Gewichte (Modellparameter) mit entsprechender *learning rate*.

In [ ]:
import torch.optim as optim
optimizer = optim.Adam(model.parameters(), lr=0.001)

Als *loss function* für Multiclass Klassifikation eignet sich die *CrossEntropyLoss*, analog zur *BinaryCrossEntropy* für binäre Klassifikation.

In [ ]:
loss_fn = nn.CrossEntropyLoss()

Die ``nn.CrossEntropyLoss`` Funktion hat eine wichtige Besonderheit:

Sie erwartet als Argumente die Vorhersage des Neuronalen Netzes und die Zielklasse, um den Fehler zu berechnen.

Dabei ist das Ergebnis des neuronalen Netzes als unnormalisierter Vektor bestehend aus den Bewertungen für die einzelnen Klassen einzubringen. Die Normalisierung entsprechend der *LogSoftmax* Aktivierungsfunktion ist dabei Bestandteil der *CrossEntropyLoss* Funktion und muss (in der Trainingsphase) vom *forward pass* des neuronalen Netzes ausgenommen werden.

Die erwartete Zielklasse wird als ein skalarer Wert übergeben, der dem Klassenindex entspricht.

Der Klassenindex ist dabei ein Wert im Bereich $[0, ..., numberClasses - 1]$.

In unserem speziellen Fall müssen wir also die Klassenindizes

In [ ]:
print(class_selection_idx)

umwandeln in

In [ ]:
print(list(range(len(class_selection_idx))))

**Beispiel:** Der Ausgangsvektor des neuronalen Netzes sei

In [ ]:
out = torch.tensor([[3.254, 0.252, 0.542, 6.233, 1.042]])

Der Target-Klassenindex ist

In [ ]:
t = torch.tensor([0])

Die *CrossEntropyLoss* berechnet sich zu:

In [ ]:
print(loss_fn(out, t))

In der Traininsschleife muss dies entsprechend stapelweise geschehen, also

In [ ]:
out = torch.tensor([[3.253, 6.124, 0.346, 0.446, 1.153],
                    [0.421, 5.255, 1.155, 0.421, 9.532],
                    [0.221, 0.564, 1.435, 2.351, 0.532]])
t = torch.tensor([1, 4, 3])
print(loss_fn(out, t))

#### Trainings-Schleife

Zur Visualisierung während dem Durchlauf der Trainingsschleife benötigen wir einige Hilfsobjekte:

In [ ]:
from IPython import display
from statistics import mean
loss_history = []
loss_ep = []
plt.figure(figsize = (12,8));

Die Traininsschleife iteriert wie gehabt über verschiedene Epochen und innerhalb jeder Epoche über die durch den ``DataLoader`` angelieferten Stapel (*batches*).

In [ ]:
n_epochs = 50

Entsprechend der oben erläuterten Besonderheiten der *CrossEntropyLoss* Funktion muss das Target-Argument in jeder Schleifeniteration aufbereitet werden.

In [ ]:
for epoch in range(n_epochs) :
    for b, batch in enumerate(data_loader) :
        optimizer.zero_grad()
        input, target = batch
        output = model(input)
        # The target returned by the DataLoader is a tensor with the original class labels
        # For the CrossEntropyLoss function we need to map this to a 1D tensor with each element 
        # a class index in [0, ..., number_of_classes-1]
        t = torch.LongTensor(len(target))   # len(target) corresponds to the batch size
        for i, e in enumerate(target):
            t[i] = torch.tensor(class_selection_idx.index(e.item()))
        loss = loss_fn(output, t)
        loss.backward()
        optimizer.step()
        loss_ep.append(loss.item())   
        
    ## Zu Visualisierungszwecken:
    loss_history.append(mean(loss_ep))
    loss_ep = []
    display.clear_output(wait=True)
    plt.plot(loss_history)
    display.display(plt.gcf())
    display.display(print("Epoch {:2}, loss: {}".format(epoch, loss_history[-1])))

#### Evaluation des Modells

Zur Evaluation des trainierten Modells ziehen wir den Test-Datensatz heran.

Diesen bereiten wir analog zu dem Trainings-Datensatz für die Verwendung auf.

Extrahieren der Datensamples mit ausgewählten Klassen:

In [ ]:
print(class_selection)

In [ ]:
test_target_idx = [i for (i,t) in enumerate(data_test.targets) if t in class_selection_idx]

Dazu benötigen wir einen entsprechenden ``SubsetRandomSampler`` ...

In [ ]:
data_sampler_test = SubsetRandomSampler(test_target_idx)

... und einen entsprechenden ``DataLoader``.

In [ ]:
data_loader_test = DataLoader(dataset=data_test, batch_size=10, sampler=data_sampler_test)

Zur Evaluation verwenden wir die klassischen Metriken aus ``scikit-learn``.

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

Wir erstellen zwei Listen:
- ``y_test`` zur Auflistung der Zielklassen aus dem Testdatensatz (erwartete Klassifizierung)
- ``y_pred`` zur Auflistung der jeweiligen Berechnung des neuronalen Netzes

In [ ]:
y_test = []
y_pred = []

Im Training hatten wir die *LogSoftmax* Aktivierung wegen der *CrossEntropyLoss* Funktion weggelassen. Wollten wir die Wahrscheinlichkeiten der berechneten Klassenzugehörigkeiten erfahren, müssten wir die *Softmax* Funktion hier auf die Ausgabe des Modells anwenden.

Da sich die Reihenfolge, also die "Rangliste", der Klassenzugehörigkeitswahrscheinlichkeiten dadurch nicht ändert, können wir auch die Klasse mit der höchsten Bewertung ermitteln. Dies geschieht durch die ``argmax`` Funktion.

In [ ]:
l = torch.randn(1,5)
print(l)
print(l.argmax())

Die Gewinnerklassen des Modells und die Zielklassen speichern wir also in den zuvor initialisierten Listen ``y_pred`` und ``y_test``.

In [ ]:
for batch_test in data_loader_test:
    input, target = batch_test

    for t in target:
        y_test.append(class_selection_idx.index(t.item()))

    prediction = model(input)
    
    for y in prediction:
        y_pred.append(y.argmax().item())

Die beiden Listen können wir nun zur Berechnung der *Confusion Matrix* und des *Classification Report*s heranziehen.

In [ ]:
confusion = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:\n', confusion)
print('\n\nClassification Report:')
print(classification_report(y_test, y_pred, target_names=class_selection))

### Ausblick

#### Convolutional Neural Networks

In den bisherigen neuronalen Netzen haben wir das Bild in einen langen eindimensionalen Vektor konvertiert.

In einem Bild befinden sich Merkmale jedoch häufig in einem Bereich benachbarter Pixel, und zwar in zwei Dimensionen. Diese Tatsache nutzen Convolutional Neural Networks (CNNs), indem sie das Bild in seiner ursprünglichen Dimensionalität als Eingabe annehmen. In einer Reihe von *feature detection layers* werden benachbarte Pixelgruppen mit Filtern verrechnet (*convolution*) und komprimiert (*pooling*). Im Anschluss finden typischerweise mehrere *fully connected layers* als Klassifikator Anwendung. Dieser führt auf Grundlage der komprimierten Merkmale die Klassifizierung durch.

Hier ein Beispiel wie ein Convolutional Neural Network mit PyTorch definiert werden kann.

In [ ]:
class ConvNet(nn.Module):
    
    def __init__(self):
        super(ConvNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 12, kernel_size=6, stride=2, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(12, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.AdaptiveMaxPool2d((6, 6))
        )
        self.classifier = nn.Sequential(
            nn.Linear(64*6*6, 1024),
            nn.ReLU(),
            nn.Linear(1024, 1024),
            nn.ReLU(),
            nn.Linear(1024, n_output)
        )
        
    def forward(self, input):
        x = self.features(input)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [ ]:
model = ConvNet()

Nach Änderung des Modells muss der Optimierer mit den neuen Modellparametern vertraut gemacht werden:

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.001)

Für eine weitreichende Erläuterung des Prinzips betrachten Sie bitte die einschlägige Literatur und die PyTorch Dokumentation.